# 主题 ：捕获类属性的定义顺序


解决问题： 想自动记录一个类中属性和方法定义的顺序，然后利用其来做一些操作（序列化、映射数据库等）

In [54]:
from collections import OrderedDict

class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected' + str(self._expected_type))
        instance.__dict__[self._name] = value
        
        
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str
    
###########################################################################
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()

### \__prepare\__ :
 Python3x 可以使用此属性为新的类名称空间创建字典

### OrderedDict:
python中的字典是无序的，因为它是按照hash来存储的，但是python中有个模块collections(英文，收集、集合)，里面自带了一个子类
OrderedDict，实现了对字典对象中元素的排序

In [55]:
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)
    
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [56]:
s = Stock('GOOG', 100, 490.1)

In [57]:
s.name

'GOOG'

In [58]:
s.as_csv()

'490.1,GOOG,100'

In [59]:
Stock("Apple", 'a lot', "500")

TypeError: Expected<class 'int'>

In [67]:
# Extend from __prepare__

from collections import OrderedDict

class NoDupOrderedDict(OrderedDict):
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()
        
    def __setitem__(self, name, value):
        if name in self:
            raise TypeError('{} already defined in {}'.format(name, self.clsname))
        super().__setitem__(name, value)
        
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_']
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return NoDupOrderedDict(clsname)
        

In [68]:
class A(metaclass=OrderedMeta):
    def spam(self):
        pass
    def spam(self):
        pass

TypeError: spam already defined in A